In [10]:
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf

from tensorflow import keras
from keras_tuner import RandomSearch
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential

In [11]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
with tf.device('/device:CPU:0'):

    # Open velocity data set
    ds = xr.open_dataset('/Users/smata/Downloads/processedData/temperatureData_30min.nc')

    # Reformat to Pandas dataframe
    df = ds['t'].to_dataframe().reset_index()

    df = df.pivot(index = 'time', columns = 'height', values = 't')

    df.columns = [f't_{int(height)}m' for height in df.columns]

    df.insert(0, 'L',    ds.L.values)
    df.insert(0, 'TKE',  ds.TKE.values)

    df.insert(0, 'hour_sin', np.sin(2 * np.pi * ds.hr_day.values / 24))
    df.insert(0, 'hour_cos', np.cos(2 * np.pi * ds.hr_day.values / 24))

    df.insert(0, 'day_sin', np.sin(2 * np.pi * ds.day_yr.values / 365))
    df.insert(0, 'day_cos', np.cos(2 * np.pi * ds.day_yr.values / 365))

    # Standardize data
    windCols      = [col for col in df.columns if col.startswith('t_')]
    stabilityCols = ['TKE', 'L']

    # Define and extract input and output columns
    inputs = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'TKE', 'L', 't_10m']

    X = df[inputs].values
    y = df[windCols].values

    # Create input and output arrays
    split_index = int(0.8 * len(X))

    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Define the model-building function
    def build_model(hp):
        model = keras.Sequential()
        
        # Choose number of layers
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(keras.layers.Dense(
                units = hp.Int(f'units_{i}', min_value = 32, max_value = 512, step = 32),
                activation = 'relu'))
        
        # Output layer
        model.add(keras.layers.Dense(y_train.shape[1]))  # Output size should match the number of wind components
        
        # Compile model
        model.compile(
            optimizer = keras.optimizers.Adam(
                hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
            ),
            loss = 'mean_squared_error',
            metrics = ['mae'])
        
        return model

    # Initialize the Keras Tuner
    tuner = RandomSearch(
        build_model,
        objective    = 'val_mae',
        max_trials   = 10,
        executions_per_trial = 3,
        directory    = 'hyperparameter_tuning',
        project_name = 'wind_forecast_nn'
    )

    # Split the training data into training and validation sets
    X_train, X_val = X_train[:int(len(X_train) * 0.8)], X_train[int(len(X_train) * 0.8):]
    y_train, y_val = y_train[:int(len(y_train) * 0.8)], y_train[int(len(y_train) * 0.8):]

    # Perform hyperparameter tuning
    tuner.search(X_train, y_train, epochs = 50, validation_data = (X_val, y_val))

    # Get the optimal hyperparameters
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials = 1)[0]

    # Print the best hyperparameters
    print("Optimal number of layers:", best_hyperparameters.get('num_layers'))
    print("Optimal learning rate:", best_hyperparameters.get('learning_rate'))
    for i in range(best_hyperparameters.get('num_layers')):
        print(f"Optimal units in layer {i}: {best_hyperparameters.get(f'units_{i}')}")

Metal device set to: Apple M3 Pro

systemMemory: 36.00 GB
maxCacheSize: 13.50 GB


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
1                 |1                 |num_layers
192               |192               |units_0
0.01              |0.01              |learning_rate

Epoch 1/50


2024-08-11 17:59:24.807523: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-11 17:59:24.807694: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-08-11 17:59:25.000941: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


156/156 [==============================] - 0s 816us/step - loss: 2911.1951 - mae: 22.4399 - val_loss: 12.0355 - val_mae: 2.7114
Epoch 2/50
156/156 [==============================] - 0s 563us/step - loss: 8.2371 - mae: 2.1764 - val_loss: 10.6041 - val_mae: 2.5404
Epoch 3/50
156/156 [==============================] - 0s 533us/step - loss: 7.4261 - mae: 2.0640 - val_loss: 10.5183 - val_mae: 2.5618
Epoch 4/50
156/156 [==============================] - 0s 547us/step - loss: 7.3667 - mae: 2.0581 - val_loss: 10.9354 - val_mae: 2.6266
Epoch 5/50
156/156 [==============================] - 0s 555us/step - loss: 7.3364 - mae: 2.0555 - val_loss: 10.9069 - val_mae: 2.6147
Epoch 6/50
156/156 [==============================] - 0s 550us/step - loss: 7.1608 - mae: 2.0216 - val_loss: 10.5763 - val_mae: 2.5760
Epoch 7/50
156/156 [==============================] - 0s 585us/step - loss: 7.1772 - mae: 2.0225 - val_loss: 10.1386 - val_mae: 2.4894
Epoch 8/50
156/156 [==============================] - 0s 530us

KeyboardInterrupt: 